In [100]:
#Load packages
import sys
import os
import requests
import re
import string
import time
import datetime

import pandas as pd

from lxml import html
from lxml import etree

import numpy as np

In [62]:
def flatten(l):
    return np.concatenate(l)

In [98]:
#Load data collected at Booth
info_2015 = pd.read_csv("../training_data/Info_2015.csv", delimiter="\t")
info_2022 = pd.read_csv("../training_data/Info_2022.csv", delimiter="\t")

#Create a dataframe for webpage links for which we need to extract nodes and xpaths

links=[]
links.append(list(set(info_2015['snapshot_link'])))
links.append(list(set(info_2022['snapshot_link'])))

links = flatten(links)
print(links)


['https://web.archive.org/web/20150217144133/https://www.apartments.com/chicago-il/', 'https://web.archive.org/web/20220612023135/https://www.apartments.com/chicago-il/', 'https://web.archive.org/web/20220321160908/https://www.apartments.com/chicago-il/', 'https://web.archive.org/web/20220121035604/https://www.apartments.com/chicago-il/', 'https://web.archive.org/web/20220426234339/https://www.apartments.com/chicago-il/', 'https://web.archive.org/web/20220403132436/https://www.apartments.com/chicago-il/']


In [105]:
# print("Start Time", start_requests)

for i, link in enumerate(links):
    print(i, str(link))
    start_requests = datetime.datetime.now()    
    try:
        response = requests.get(str(link), proxies={'http': '222.255.169.74:8080'}, timeout=5)
#         print(response.status_code)
    except requests.exceptions.Timeout:
        print('The request timed out')

    root = html.fromstring(response.text)
    tree = root.getroottree()

    parents_blacklist=['[document]','html','head',
                       'style','script','body',
                       'section','tr',
                       'td','label','ul','header',
                       'aside']

    regexpNS = "http://exslt.org/regular-expressions"
    finds = root.xpath("//*[re:test(., '[a-zA-Z0-9]', 'i')]", namespaces={'re':regexpNS})

    total_nodes = 1
    nodes = []
    xpaths = []

    start = datetime.datetime.now()

    for i, find in enumerate(finds):
        path = tree.getpath(find)
        if i == len(finds) - 1:
            text = tree.xpath(path)[0].text
            if text != None and len(text) > 0 and len(text) < 60 and text != " ":
                total_nodes = total_nodes + 1
                nodes.append(text.strip())
                xpaths.append(path)
                break
        else:
            path_next = tree.getpath(finds[i+1])         
            if path in path_next:
                continue
            else:
                text = tree.xpath(path)[0].text
                if text != None and len(text) > 0 and len(text) < 60 and text != " ":
    #                 print(text.strip(), path)
                    total_nodes = total_nodes + 1
                    nodes.append(text.strip())
                    xpaths.append(str(path))

    # print("\n \n Total Number of Nodes Are:", total_nodes) 
    # print(nodes)
    # print(xpaths)
    dataset = pd.DataFrame({"nodes": nodes, "xpaths":xpaths})
    file_name = str(link[28:42]) + "_nodes_xpaths.csv"
    file_path = os.path.join("../training_data/nodes_xpaths", file_name)
    dataset.to_csv(file_path, sep='\t')   


0 https://web.archive.org/web/20150217144133/https://www.apartments.com/chicago-il/
1 https://web.archive.org/web/20220612023135/https://www.apartments.com/chicago-il/
2 https://web.archive.org/web/20220321160908/https://www.apartments.com/chicago-il/
3 https://web.archive.org/web/20220121035604/https://www.apartments.com/chicago-il/
4 https://web.archive.org/web/20220426234339/https://www.apartments.com/chicago-il/
5 https://web.archive.org/web/20220403132436/https://www.apartments.com/chicago-il/
